In [25]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [26]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [27]:
%matplotlib widget

In [28]:
%reload_ext autoreload
%autoreload 2

In [29]:
## force template
dist = 10.39
z_sep = -15.85
y_offset = 4.9
test = force_vs_time(dist,z_sep,200,3,"z",20, yuk_or_grav="yuk",offset_y=y_offset, 
                     alpha=1e8)
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = np.sqrt(2)*fft_norm(50000, 5000)

_,ax = plt.subplots()
x = list(test[1])*10
fft1 = np.abs(np.fft.rfft(x))*scale/np.sqrt(10)
fft_angles = np.angle(np.fft.rfft(x))
freq1 = np.fft.rfftfreq(len(x), d=1./5000)
ax.semilogy(freq1, fft1, label='7.6 $\mu m$ sphere - Z=-15.7')
ax.set(xlim=(1,100), ylim=(None,None), xlabel='frequency [Hz]', ylabel='Force [N]', title=r'$\alpha=1x10^{8}$')
ax.legend()
print(fft1[freq1==12])

TypeError: can't multiply sequence by non-int of type 'float'

In [24]:
[print(fft1[freq1==freq_][0]) for freq_ in 3*np.arange(1,14)]

4.68952985727e-18
6.86879411916e-18
2.92835872514e-18
8.61637877166e-18
1.48985916779e-18
7.15587765636e-18
6.99119057336e-18
2.49970446358e-18
4.412814934e-18
1.03432982087e-17
8.87405984508e-18
8.90155956221e-18
4.82608802896e-18


[None, None, None, None, None, None, None, None, None, None, None, None, None]

In [110]:
def func_psd(i):
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Discharge/New_Discharge/Discharge_'+str(i)+'.h5'
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking373/Shaking4_'+str(i)+'.h5'

    bb = BeadDataFile.BeadDataFile(fname=fname)
    fft_ = bb.psd2('z')[1]
    return (fft_)

In [111]:
test_norms3 = np.array(Parallel(n_jobs=20)(delayed(func_psd)(i) for i in tqdm(range(5000))))




  0%|          | 0/5000 [00:00<?, ?it/s]


  0%|          | 20/5000 [00:01<04:28, 18.57it/s]


  1%|          | 40/5000 [00:01<03:34, 23.16it/s]


  1%|          | 60/5000 [00:02<03:21, 24.55it/s]


  2%|▏         | 80/5000 [00:02<02:40, 30.72it/s]


  2%|▏         | 100/5000 [00:02<02:08, 38.20it/s]


  2%|▏         | 120/5000 [00:02<01:48, 44.99it/s]


  3%|▎         | 140/5000 [00:03<01:36, 50.13it/s]


  3%|▎         | 160/5000 [00:03<01:26, 56.17it/s]


  4%|▎         | 180/5000 [00:03<01:17, 62.17it/s]


  4%|▍         | 200/5000 [00:03<01:13, 65.14it/s]


  4%|▍         | 220/5000 [00:04<01:12, 65.66it/s]


  5%|▍         | 240/5000 [00:04<01:08, 69.91it/s]


  5%|▌         | 260/5000 [00:04<01:06, 71.63it/s]


  6%|▌         | 280/5000 [00:04<01:01, 76.60it/s]


  6%|▌         | 300/5000 [00:05<01:04, 73.21it/s]


  6%|▋         | 320/5000 [00:05<01:06, 70.11it/s]


  7%|▋         | 340/5000 [00:05<01:03, 73.78it/s]


  7%|▋         | 360/5000 [00:06<01:06, 69.96it/s]


  8%

In [112]:
i = 100
fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
# fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking373/Shaking4_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
dist = 392.4-np.mean(bb.cant_pos[0])*50-7.6/2
freq_psd = bb.psd2('z')[0]
shaking3_psd = np.mean(test_norms, axis=0)
noshaking1_psd = np.mean(test_norms2, axis=0)
shaking4_psd = np.mean(test_norms3, axis=0)

In [8]:
z_scale2

array([  3.99023794e-20])

In [7]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/New_Discharge/',
                         start_file=0, max_file=1000)
bdf_z = bdf_discharge[:5]

freq = bdf_z[0].psd2('z')[0]
z_scale2 = bdf_z[0].psd2('z')[1][freq==71]
for bb in bdf_z:
    z_scale2 += bb.psd2('z')[1][freq==71]
z_scale2 /= len(bdf_z)

charges = 7
force = charges * 1.6e-19 * 20 / 8.6e-3 * 0.63 # in Newtons
z_scale2 = force/np.sqrt(z_scale2)

No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
No laser power data
15  files in folder
15  files loaded


In [85]:
shaking3_psd.shape, correction.shape

((25001,), (25001,))

In [162]:
_,ax = plt.subplots(figsize=(9,4))
ax.semilogy(freq_psd, np.sqrt(shaking3_psd)*z_scale2, label='Wilson 378-3')
ax.semilogy(freq_psd, 0.9*np.sqrt(shaking4_psd)*z_scale2, label='Wilson 373-4')
ax.semilogy(freq_psd, 1.03*np.sqrt(noshaking1_psd)*z_scale2+fft1*np.sqrt(correction)*3, '--', label=r'noise+signal[$\alpha=10^8$]', alpha=0.8)
ax.semilogy(freq_psd, 1.03*np.sqrt(noshaking1_psd)*z_scale2, 'r--', label=r'noise only', alpha=0.8)
ax.legend()
_ = ax.set(xlim=(5,50), ylim=(1e-17,2e-16), xlabel='Frequency [Hz.]', ylabel='Force [N]', title=r'Z force')
ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [72]:
fft_tf[freq_tf==2]

array([ 0.02537228])

In [13]:
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
_,ax = plt.subplots(figsize=(9,4))
freq_tf, fft_tf = tf_z.psd2('z')
fft_tf /= fft_tf[freq_tf==71]
freq_corr = np.arange(1,100)
value_corr = [fft_tf[freq_tf==freq_] for freq_ in freq_corr]
correction = np.interp(freq_psd, freq_corr, np.array(value_corr).flatten())
ax.semilogy(freq_tf, np.sqrt(fft_tf))
ax.semilogy(freq_psd, np.sqrt(correction), '--')
_ = ax.set(xlim=(1,100), xlabel='Frequency [Hz.]', ylabel='Response [a.u]', title=r'Z force - TF')

No height data
No cantilever data
Error loading spinning data
No laser power data


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'freq_psd' is not defined

### Phase

In [73]:
drive_freq = 33

def func_filter(i):
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Discharge/New_Discharge/Discharge_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking373/Shaking4_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200924/Bead1/Shaking/Shaking345/SpinShaking0_'+str(i)+'.h5'


    bb = BeadDataFile.BeadDataFile(fname=fname)
    res = bb.response_at_freq2('x', drive_freq=drive_freq, bandwidth=2)
    return (res)

In [74]:
test_phases = np.array(Parallel(n_jobs=40)(delayed(func_filter)(i) for i in tqdm(range(10000))))


100%|██████████| 10000/10000 [31:43<00:00,  5.66it/s]

In [75]:
x = list(test[1])*10
fft_angles = np.angle(np.fft.rfft(x))
freq1 = np.fft.rfftfreq(len(x), d=1./5000)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.x2))
freq2 = np.fft.rfftfreq(len(tf_z.x2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

No height data
No cantilever data
Error loading spinning data
No laser power data


In [76]:
phase_th = fft_angles[freq1==drive_freq]  ## theory 
phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)

In [77]:
ll = likelihood_analyser.LikelihoodAnalyser()
phases = []
amps = []
fit_kwargs = {'A': 1, 'f':drive_freq, 'phi': np.pi,
          'error_A': 1, 'error_phi': 0.5, 'error_f': 1, 'errordef': 1,
          'limit_phi': [0, 2 * np.pi],
          'limit_A': [-1000, 1000],
          'print_level': 0, 'fix_A': False, 'fix_phi': False, 'fix_f': True}
for i in range(100):
    tmp_avg = np.mean(test_phases[100*i:(i+1)*100], axis=0)
    mm = ll.find_mle_sin(tmp_avg[5000:-5000]*10000, **fit_kwargs)
    print(mm.values)
    phases.append(mm.values[2])
    amps.append(mm.values[0])

<ValueView of Minuit at 13c11268>
  A: 0.1092168224447505
  f: 33.0
  phi: 2.775067423422573
<ValueView of Minuit at d75d598>
  A: 0.30208508392058775
  f: 33.0
  phi: 0.4419222469138637
<ValueView of Minuit at 13c33ba8>
  A: 0.9464922814921692
  f: 33.0
  phi: 0.16742307881311874
<ValueView of Minuit at 3d534b8>
  A: 1.3344224742475035
  f: 33.0
  phi: 0.16566846633026724
<ValueView of Minuit at 597ea58>
  A: 1.5371586810392728
  f: 33.0
  phi: 0.16893317575750422
<ValueView of Minuit at 13baa7d8>
  A: 1.710672700368832
  f: 33.0
  phi: 0.20573866732490972
<ValueView of Minuit at 39de728>
  A: 1.3312245670260836
  f: 33.0
  phi: 0.17431947013528462
<ValueView of Minuit at 14cd9138>
  A: 0.9609550792455366
  f: 33.0
  phi: 0.1774492966163746
<ValueView of Minuit at 3c42588>
  A: 0.6094700563481865
  f: 33.0
  phi: 0.16201958975418757
<ValueView of Minuit at d765798>
  A: 0.12589220708923676
  f: 33.0
  phi: 6.013118811071247
<ValueView of Minuit at 13b94ef8>
  A: 0.23158915441399586
  

<ValueView of Minuit at 13c3f3f8>
  A: 0.04304546925345676
  f: 33.0
  phi: 1.4505592569955443
<ValueView of Minuit at 495f318>
  A: 0.032960799109673644
  f: 33.0
  phi: 2.1215824616320402
<ValueView of Minuit at 4d3b4f8>
  A: 0.187441539931001
  f: 33.0
  phi: 0.22820777081789845
<ValueView of Minuit at 39de728>
  A: 0.0991599781767718
  f: 33.0
  phi: 3.309648538635992
<ValueView of Minuit at d765798>
  A: 0.2706120307211677
  f: 33.0
  phi: 0.17483125665297636
<ValueView of Minuit at 14cd9138>
  A: 0.568408547225772
  f: 33.0
  phi: 3.3532064065072746
<ValueView of Minuit at d75d598>
  A: 0.13407034721706168
  f: 33.0
  phi: 3.515703243139299
<ValueView of Minuit at 13c33ba8>
  A: 0.5314411528771643
  f: 33.0
  phi: 3.3856704574604772
<ValueView of Minuit at 3d534b8>
  A: 0.6161686239103119
  f: 33.0
  phi: 3.317049625876086
<ValueView of Minuit at 597ea58>
  A: 0.767948250024233
  f: 33.0
  phi: 3.3387508926907805
<ValueView of Minuit at 13baa7d8>
  A: 0.24489185065351649
  f: 33.

In [82]:
_,ax = plt.subplots(figsize=(9,4.5))
ax.scatter(np.arange(0,10000,100)*10/60, phases, label='measured phase')
ax.plot(np.arange(0,10000,100)*10/60, np.arange(100)*0+phase_all, 'r--', label='theory-pull')
ax.plot(np.arange(0,10000,100)*10/60, np.arange(100)*0+phase_all-np.pi, 'g--', label='theory-push')
ax.legend()
ax.set(ylabel='Phase', xlabel='Time [minute]', title=str(drive_freq)+' Hz.')

ax.grid(True)
ax.yaxis.set_major_locator(plt.MultipleLocator(np.pi / 2))
ax.yaxis.set_minor_locator(plt.MultipleLocator(np.pi / 12))
ax.yaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [211]:
bandwidth = 2
fsamp = 5000
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
responsefilt = signal.filtfilt(b, a, x)
mm = ll.find_mle_sin(responsefilt[5000:-5000], **fit_kwargs)
phase_th = mm.values[2]

In [16]:
import numpy as np
import matplotlib.pyplot as plt

def multiple_formatter(denominator=2, number=np.pi, latex='\pi'):
    def gcd(a, b):
        while b:
            a, b = b, a%b
        return a
    def _multiple_formatter(x, pos):
        den = denominator
        num = np.int(np.rint(den*x/number))
        com = gcd(num,den)
        (num,den) = (int(num/com),int(den/com))
        if den==1:
            if num==0:
                return r'$0$'
            if num==1:
                return r'$%s$'%latex
            elif num==-1:
                return r'$-%s$'%latex
            else:
                return r'$%s%s$'%(num,latex)
        else:
            if num==1:
                return r'$\frac{%s}{%s}$'%(latex,den)
            elif num==-1:
                return r'$\frac{-%s}{%s}$'%(latex,den)
            else:
                return r'$\frac{%s%s}{%s}$'%(num,latex,den)
    return _multiple_formatter

class Multiple:
    def __init__(self, denominator=2, number=np.pi, latex='\pi'):
        self.denominator = denominator
        self.number = number
        self.latex = latex

    def locator(self):
        return plt.MultipleLocator(self.number / self.denominator)

    def formatter(self):
        return plt.FuncFormatter(multiple_formatter(self.denominator, self.number, self.latex))